In [2]:
import os

In [3]:
%pwd

'f:\\MlProject\\Datascienceproject\\research'

In [4]:
os.chdir("../")
%pwd

'f:\\MlProject\\Datascienceproject'

In [47]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [48]:
from src.datascience.constants.__init__ import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion  # Fixed typo

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,  # Fixed typo
            unzip_dir=config.unzip_dir  # Fixed typo
        )

        return data_ingestion_config


In [50]:
import os
import zipfile
from urllib.request import urlretrieve
from src.datascience import logger

In [51]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info:\n{headers}")

        else:
            file_size = os.path.getsize(self.config.local_data_file) / (1024 * 1024)  # Convert bytes to MB
            logger.info(f"File already exists with size: {file_size:.2f} MB")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted files to {unzip_path}")



In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-11 13:32:12,000: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-11 13:32:12,004: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 13:32:12,008: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-11 13:32:12,011: INFO: common: created directory at: artifacts]
[2025-03-11 13:32:12,014: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-11 13:32:12,831: INFO: 3968078424: artifacts/data_ingestion/data.zip downloaded with the following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0EB4:359F:F5239:27E292:67CFED54
Accept-Ranges: bytes
Date: Tue, 1